In [1]:
import sys

# --- Instalação das bibliotecas ---
try:
    import pandas as pd
except ImportError:
    print("Instalando pandas...")
    !{sys.executable} -m pip install pandas
    import pandas as pd

try:
    import s3fs
except ImportError:
    print("Instalando s3fs...")
    !{sys.executable} -m pip install s3fs
    import s3fs
# --- Fim da instalação ---

# Lista dos 7 arquivos específicos no S3
json_files = [
    's3a://bucket-raw-upa-connect/dados_2025_05_20.json',
    's3a://bucket-raw-upa-connect/dados_2025_05_21.json',
    's3a://bucket-raw-upa-connect/dados_2025_05_22.json',
    's3a://bucket-raw-upa-connect/dados_2025_05_23.json',
    's3a://bucket-raw-upa-connect/dados_2025_05_24.json',
    's3a://bucket-raw-upa-connect/dados_2025_05_25.json',
    's3a://bucket-raw-upa-connect/dados_2025_05_26.json'
]

# Caminho de saída
s3_output_path = 's3a://bucket-raw-upa-connect/tabela_historico_sensor.csv'

def process_selected_json_to_csv(json_file_list, csv_output_path):
    """
    Lê arquivos JSON específicos no S3, une-os,
    ordena pela coluna 'data_hora' e salva como CSV no S3.
    """
    fs = s3fs.S3FileSystem()

    if not json_file_list:
        print("A lista de arquivos JSON está vazia.")
        return

    all_dataframes = []

    print(f"Lendo {len(json_file_list)} arquivos JSON do S3...")
    for file_path in json_file_list:
        try:
            with fs.open(file_path, 'rb') as f:
                df = pd.read_json(f)
                all_dataframes.append(df)
        except Exception as e:
            print(f"Erro ao ler o arquivo {file_path}: {e}")
            continue

    if not all_dataframes:
        print("Nenhum dado foi carregado com sucesso. Verifique os arquivos JSON.")
        return

    # Concatena todos os DataFrames
    combined_df = pd.concat(all_dataframes, ignore_index=True)

    # Converte a coluna 'data_hora' para datetime e ordena
    print("Convertendo 'data_hora' e ordenando os dados...")
    combined_df['data_hora'] = pd.to_datetime(combined_df['data_hora'])
    combined_df = combined_df.sort_values(by='data_hora').reset_index(drop=True)

    # Salva o DataFrame combinado como CSV no S3
    try:
        print(f"Salvando o arquivo CSV consolidado em {csv_output_path}...")
        with fs.open(csv_output_path, 'wb') as f:
            combined_df.to_csv(f, index=False)
        print("Processo concluído com sucesso!")
    except Exception as e:
        print(f"Erro ao salvar o arquivo CSV no S3: {e}")
        print("Verifique suas permissões de escrita para o bucket S3.")

# Executa o processo com os arquivos especificados
process_selected_json_to_csv(json_files, s3_output_path)

Lendo 7 arquivos JSON do S3...
Convertendo 'data_hora' e ordenando os dados...
Salvando o arquivo CSV consolidado em s3a://bucket-raw-upa-connect/tabela_historico_sensor.csv...
Processo concluído com sucesso!
